In [ ]:
"""Import all the necessary libraries"""
import os
from os.path import join
from os.path import join as opj
import glob
from pathlib import Path
from pprint import pprint
from datetime import datetime, timedelta

#Import form OST tool box
from ost import Sentinel1_GRDBatch
from ost.helpers import vector, raster

#Sheduling Tasks
from apscheduler.scheduler import Scheduler

import rasterio
from ost.helpers import helpers as h, raster as ras


#Importing Flood Processing Module
from Flood_OST_S1 import Sentinel1Flood

In [ ]:
#Defining the main working directory
path = "D:\\2020\\46_phil_storm\\z_results"


#Defining the Area of interest as a point
lat, lon =    17.76,121.72


aoi = 'POINT ({} {})'.format(lon, lat)
aoi = vector.latlon_to_wkt(lat, lon, buffer_meter=40000, envelope=True)

#Defining the Area as a bounding box
"""aoi = 'POLYGON((89.058055555556 24.277222222222,\
                       92.580277777778 24.277222222222,\
                       92.580277777778 23.373055555556,\
                       89.058055555556 23.373055555556,\
                       89.058055555556 24.277222222222))'"""


# Time of interest - pre disaster
start_pre = '2020-11-04'
end_pre = '2020-11-05'

# Activation occurrence date
start_post = '2020-11-15'
# End Activation date --- eg: current date
end_post = '2020-11-16'

#Required post date time formatting
t1 = start_post
y1=t1[0:4]
m1=t1[5:7]
d1=t1[-2:]

date1 =m1+"-"+d1+"-"+y1
d1 = datetime.strptime(date1,'%m-%d-%Y').date()

#Create a processing directory
project_dir_pre = join(path, 'phil_s1_idc_16', 'pre')
project_dir_post = join(path, 'phil_s1_idc_16', 'post')


In [ ]:
#Creationa of GRD batch processing instances for pre and post scenarios
s1_grd_pre = Sentinel1_GRDBatch(
    project_dir=project_dir_pre,
    aoi = aoi,
    start = start_pre,
    end = end_pre
)


#ARD parameter defining and Processing of the data
#ARD parameters pre images
s1_grd_pre.ard_parameters['single ARD']['resolution'] = 30
s1_grd_pre.ard_parameters['single ARD']['product type'] = 'GTC-sigma0'
s1_grd_pre.ard_parameters['single ARD']['create ls mask'] = False
s1_grd_pre.ard_parameters['single ARD']['dem']['image resampling'] = 'BILINEAR_INTERPOLATION'
s1_grd_pre.ard_parameters['single ARD']['dem']['dem name']= 'SRTM 3Sec'
s1_grd_pre.ard_parameters['single ARD']['to db'] = 'True'
s1_grd_pre.ard_parameters['single ARD']['polarisation'] = 'VV'



Basic Purpose of below mentioned functions
1. Tranfering .img to .tiff
2. Download data and definition of process workflow.
3. Search Latest Post image file



In [ ]:
# (1)Function for convert .img data >> .ge0tiff    
def ard_to_gtiff(infile, outfile, driver='GTiff', to_db=True):    
    prefix = glob.glob(os.path.abspath(infile[:-4]) + '*data')[0]
    if len(glob.glob(opj(prefix, '*VV*.img'))) == 1:
        co_pol = glob.glob(opj(prefix, '*VV*.img'))[0]   
    with rasterio.open(co_pol) as co:        
        # get meta data
        meta = co.meta
        # update meta
        meta.update(driver=driver, count=1, nodata=0)
        with rasterio.open(outfile, 'w', **meta) as dst:
            for i, window in co.block_windows(1):                
            # read arrays and turn to dB (in case it isn't)
                co_array = co.read(window=window)
                for k, arr in [(1, co_array)]:
                    dst.write(arr[0, ], indexes=k, window=window)   

In [ ]:
# (2)This function fill find the corresponding pre image to the post image and download both pre,post images                
def process_downloaded_data():


    #Latest Post Sentinel Image/s
    latestDf_grd_post = iDf_grd_post[iDf_grd_post.acquisitiondate == iDf_grd_post.acquisitiondate.min()]
                
    #Finding the Orbital Corresponding Pre Image
    s1_grd_pre.search()
    iDf_grd_pre= s1_grd_pre.inventory.copy()
    orbit_grd_pre =  iDf_grd_pre.loc[iDf_grd_pre['relativeorbit'].isin(np.array(latestDf_grd_post.relativeorbit.unique()))]
    Df_grd_pre = orbit_grd_pre[orbit_grd_pre.acquisitiondate == orbit_grd_pre.acquisitiondate.max()]
                        
    #Download
    s1_grd_pre.download(Df_grd_pre,mirror= '1')
    s1_grd_post.download(latestDf_grd_post,mirror= '1')
    print(' INFO: Latest scene found for {}'.format(latestDf_grd_post['acquisitiondate'].values[0]))
    s1_grd_post.plot_inventory(latestDf_grd_post, transparency=.5)
    print(' INFO: Pre scene found for {}'.format(Df_grd_pre['acquisitiondate'].values[0]))
    s1_grd_pre.plot_inventory(Df_grd_pre, transparency=.5)
    
    #Process of the downloaded images
    b1 = datetime.datetime.now()

    s1_grd_pre.grds_to_ard(
        inventory_df=Df_grd_pre, 
        timeseries=False, 
        timescan=False, 
        mosaic=True, 
        overwrite=True, 
        subset=None)
    s1_grd_post.grds_to_ard(
        inventory_df=latestDf_grd_post, 
        timeseries=False, 
        timescan=False, 
        mosaic=True, 
        overwrite=True, 
        subset=None)

    b2 = datetime.datetime.now()-b1
    print("Process execution time {}".format(b2))
    
    #getting the data from dim outs puts. need to refine
    orbit_post = latestDf_grd_post['relativeorbit'].values[0]
    orbit_pre = Df_grd_pre['relativeorbit'].values[0]
    date_post = str(latestDf_grd_post.acquisitiondate.max())
    date_pre = str(Df_grd_pre.acquisitiondate.max())
    #.dim data path
    dim_post_dir = join(project_dir_post, 'processing', str(orbit_post), str(date_post)) 
    dim_pre_dir = join(project_dir_pre, 'processing', str(orbit_pre), str(date_pre)) 
    #making Tiff_Data in project folder
    in_proj_folder= os.path.normpath(project_dir_post + os.sep + os.pardir)
    if not os.path.exists(in_proj_folder+"\\Tiff_Data"):
        os.makedirs(in_proj_folder+"\\Tiff_Data")
    proj_tiff_folder=os.path.normpath(in_proj_folder+"\\Tiff_Data")

    final_post_path = os.path.join(dim_post_dir + "\\{}_{}.bs.data".format(date_post,orbit_post))
    final_pre_path = os.path.join(dim_pre_dir + "\\{}_{}.bs.data".format(date_pre,orbit_pre)) 

    #Converting dim to geotiff
    ard_to_gtiff(final_pre_path,proj_tiff_folder+"\\"+"pre.tiff", driver='GTiff', to_db=False)
    ard_to_gtiff(final_post_path,proj_tiff_folder+"\\"+"post.tiff", driver='GTiff', to_db=False)
    
    ## Processing Using Flood module
    # making flood result folder in project folder
    # in_proj_folder= os.path.normpath(project_dir_post + os.sep + os.pardir)
    if not os.path.exists(in_proj_folder+"\\Flood_Result"):
        os.makedirs(in_proj_folder+"\\Flood_Result")
    proj_flood_result_folder=os.path.normpath(in_proj_folder+"\\Flood_Result")

    # Data input and output
    flood_data_input = str(proj_tiff_folder)+"\\"
    flood_result_output = str(proj_flood_result_folder)+"\\"
    
    # From Flood_OST_S1 import Sentinel1Flood. This will produce flood result based on the change image threshold of -3 decibels
    global test01
    # Defining an object fro the global variables
    test01 = Sentinel1Flood(flood_data_input,flood_result_output)
    test01.change(out_change_name = "change")
    test01.thresholding(threshold_value = -3, out_thresh_name = "threshold")
    test01.maj_filtering(filter_size = 3, out_maj_name = "majority")
    test01.ras2poly(out_poly_name = "poly")
    

The below-mentioned search_latest function will search Sentinel-1 data according to the defined time and geographical requirements using the OST (Open SAR toolkit) functions.

In [ ]:
# (3)Search latest file post image
def search_latest():
    
    # Making a  post directory to update the full inventory here
    global s1_grd_post
    s1_grd_post = Sentinel1_GRDBatch(
    project_dir=project_dir_post,
    aoi = aoi,
    start = start_post,
    end = end_post)
        
    # Post Parameter definiton
    s1_grd_post.ard_parameters['single ARD']['resolution'] = 30
    s1_grd_post.ard_parameters['single ARD']['product type'] = 'GTC-sigma0'
    s1_grd_post.ard_parameters['single ARD']['create ls mask'] = False
    s1_grd_post.ard_parameters['single ARD']['dem']['image resampling'] = 'BILINEAR_INTERPOLATION'
    s1_grd_post.ard_parameters['single ARD']['dem']['dem name']= 'SRTM 3Sec'
    s1_grd_post.ard_parameters['single ARD']['to db'] = 'True'
    s1_grd_post.ard_parameters['single ARD']['polarisation'] = 'VV'    
    
    #Searching the latet post image
    s1_grd_post.search()
    #\post\inventory
    global iDf_grd_post
    inv_path = project_dir_post+'\\inventory'
    con = os.listdir(inv_path)
    if len(con) == 0:
        print("Still seraching for data")
        
    else:
            
        iDf_grd_post = s1_grd_post.inventory.copy()

        #laset observatoin data time formatting
        t2=iDf_grd_post.acquisitiondate.min()
        y2=t2[0:4]
        m2=t2[4:6]
        d2=t2[-2:]

        date2 =m2+"-"+d2+"-"+y2
        d2=datetime.datetime.strptime(date2,'%m-%d-%Y').date()
        #d2=datetime.strptime(date2,'%m-%d-%Y').date()
        print("Checked for :"+str(datetime.datetime.now()))
        if (d2 >= d1):
            #send_email1()

            global process_start_time
            process_start_time = str(datetime.datetime.now() + timedelta(seconds = 15))
            sched11.add_date_job(my_job, process_start_time, ['text'])

            sched11.start()

            sched.shutdown()    
        else:
            print("Searching the data...")
    
       

This below sheduler option will run the search_latest fuction for every time interval defined as hours.

In [ ]:
sched = Scheduler()

# Schedule job_function to be called every 10 minutes
@sched.interval_schedule(hours= 1/10)
def job_function():
    search_latest()
sched.start() 

Once the "search_latest function" satisfies, "process_downloaded_data" function will be executed.

In [ ]:
sched11 = Scheduler()
def my_job(test):
    process_downloaded_data()

Final result will be based on the threshold value of -3 decibels. After checking the result quality, to make changes 
further filtering can be done as follows. PS. test01 is the flood processing object created in the 
process_downloaded_data fucntion.


In [ ]:
#Filter the flood result if needed
#test01 = Sentinel1Flood(flood_data_input,flood_result_output) -- already made in previous steps
#test01.change(out_change_name = "change") -- already made in previous steps
#Process in the oder of (thresh - maj -- ras) then all the pre created result files will be overwritten.
"""test01.thresholding(threshold_value = -5, out_thresh_name = "threshold")
test01.maj_filtering(filter_size = 3, out_maj_name = "majority")
test01.ras2poly(out_poly_name = "poly")"""